In [1]:
import sys

import numpy as np
import math
from scipy.sparse import csr_matrix
from numpy import linalg as LA
from scipy import linalg as SLA

# python3 -m pip install pyopengl
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

# python3 -m pip install pygame
import pygame
from pygame.locals import *

from draw_object import ConfigureEnv

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class CartPoleState(object):
    def __init__(self):   
        self.x=0.
        self.x1=0.
        self.th=.2
        self.th1=0. 

        # init constants
        self.tau = 1/60.
        Mp = 1
        Mc = 1
        self.l = 1
        self.c1 = 1/(Mp+Mc)
        self.c2 = self.l*Mp/(Mp+Mc)
        self.g = 9.8
        
        """change the dynamicsNoise here"""
        self.dynamicsNoise = 0.01
    
    def step(self, u):
        the2 = self.g*math.sin(self.th) \
               + math.cos(self.th)*(-self.c1*u-self.c2*self.th1*self.th1*math.sin(self.th))
        the2 /= self.l*4/3 - self.c2*math.cos(self.th)*math.cos(self.th)
        x2 = self.c1*u + self.c2*(self.th1*self.th1*math.sin(self.th) - the2*math.cos(self.th))
        
        self.x   = self.x + self.tau*self.x1
        self.x1  = self.x1 + self.tau*x2
        self.th  = self.th + self.tau*self.th1
        self.th1 = self.th1 + self.tau*the2
        
        if self.dynamicsNoise:
            self.x1 += self.dynamicsNoise*np.random.normal(0,1)
            self.th1 += self.dynamicsNoise*np.random.normal(0,1)


In [4]:
def draw_objects(env, x, theta):
    # draw guide lines
    env.draw_line()  

    # cart
    glColor3f(1,1,1); 
    glPushMatrix()
    glTranslatef(x, 0., 0.)
    env.draw_cube(1., 0.2, 0.2)  
    env.draw_line2()
    
    # pole
    glColor3f(1,0,0); 
    glPushMatrix()
    glRotatef(theta*180./math.pi, 0., 1., 0.)
    glTranslatef(0, 0., 0.5)
    env.draw_cube(0.1, 0.1, 1.)  
    
    glPopMatrix()
    glPopMatrix()
    

In [5]:
def testMove(s):
    """implement the controller gains here"""
    # K = np.zeros(4)
    # u = K[0]*s.x + K[1]*s.x1 + K[2]*s.th + K[3]*s.th1;
    nonzero_A = np.array([1, -s.c2 * s.g/(4/3. - s.c2), s.g/(4/3. - s.c2), 1])
    col_A = np.array([1,2,2,3])
    row_A = np.array([0,1,3,2])
    A = csr_matrix((nonzero_A, (row_A, col_A)),shape=(4,4),dtype=np.float32).toarray()
    # print(A)
    B = np.array([0,s.c1+s.c1*s.c2/(4/3. - s.c2),0,-s.c1/(4/3. - s.c2)])
    B = np.expand_dims(B, axis=-1)
    Q = np.diag([1,0,1,0])
    R = np.eye(1)
    K = -B.transpose()@SLA.solve_continuous_are(A,B,Q,R)

    x_ = np.array([s.x, s.x1, s.th, s.th1])
    x_ = np.expand_dims(x_, axis=-1)
    print(f'shape of K is {K.shape}')
    print(f'shape of x_ is {x_.shape}')
    # u = K@x_
    # print(u)
    # u = riccati_numeric() @ x_
    u = K[0][0]*s.x + K[0][1]*s.x1 + K[0][2]*s.th + K[0][3]*s.th1
    s.step(u)
    
    # return translation x and rotation theta
    return s.x, s.th

In [6]:
def riccati_numeric():
    c1 = 0.5
    c2 = 0.5
    g = 9.8
    Q = np.diag([1,0,1,0])

    nonzero_A = np.array([1., -c2*g/(4/3. - c2), g/(4/3. - c2), 1.], dtype=np.float32)
    col_A = np.array([1,2,2,3])
    row_A = np.array([0,1,3,2])
    A = csr_matrix((nonzero_A, (row_A, col_A)),shape=(4,4),dtype=np.float32).toarray()
    # print(A)
    B = np.array([0,c1+c1*c2/(4/3. - c2),0,-c1/(4/3. - c2)])
    B = np.expand_dims(B, axis=-1)
    print(B.shape)
    print(A.shape)
    R = np.eye(1)
    P = Q.copy()
    epsilon = 1e-3
    while True:
        P_ = P  + epsilon*(A.transpose()@P + P.transpose()@A - P@B*R**-1@B.transpose()@P + Q)
        if LA.norm(P_ - P, 'fro') < 1e-4:
            break
        else:
            P = P_
    print(-B.transpose()@P)
    P_analytic = riccati_analytic(A,B,Q,R)
    print(-B.transpose()@P_analytic)
    K = -B.transpose()@P_analytic
    print(K.transpose()[0][0])

    print(f'A = {A}\n B = {B}\n A*B = {A@B}')
    print(np.linalg.eigvals(A))
    print(SLA.eigvals(A + B @ K))
    return -B.transpose()@P_analytic

def riccati_analytic(A,B,Q,R):
    return SLA.solve_continuous_are(A,B,Q,R)


riccati_numeric()

(4, 1)
(4, 4)
[[ 1.00004957  2.60827348 52.9453595  16.59403107]]
[[ 1.          2.60879093 52.94837257 16.59517603]]
1.000000000000003
A = [[ 0.    1.    0.    0.  ]
 [ 0.    0.   -5.88  0.  ]
 [ 0.    0.    0.    1.  ]
 [ 0.    0.   11.76  0.  ]]
 B = [[ 0. ]
 [ 0.8]
 [ 0. ]
 [-0.6]]
 A*B = [[ 0.8]
 [ 0. ]
 [-0.6]
 [ 0. ]]
[ 0.         0.         3.4292858 -3.4292858]
[-3.4288164 +0.09782127j -3.4288164 -0.09782127j -0.50622004+0.49342822j
 -0.50622004-0.49342822j]


array([[ 1.        ,  2.60879093, 52.94837257, 16.59517603]])

In [1]:
def main():
    pygame.init()
 
    display = (1000,750)
    pygame.display.set_mode(display, DOUBLEBUF|OPENGL, RESIZABLE)

    glRotatef(-90.,1.,0, 0)
    glScaled(0.8, 0.8, 0.8)

    env = ConfigureEnv()
    s = CartPoleState()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            env.mouseMove(event);

        glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
        
        # calculate and update opengl
        x, th = testMove(s)
        draw_objects(env, x, th)
        
        pygame.display.flip()
        pygame.time.wait(10)



In [ ]:
main()